In [1]:
import pandas as pd

data = pd.read_csv('data/kalshi_panel.csv')
ticker_max_week = data.loc[data.groupby('ticker')['week'].idxmax()].reset_index(drop=True)

In [ ]:
from ipywidgets import interact, IntSlider
import seaborn as sns
import ipywidgets as widgets

pretty_slider = widgets.IntSlider(
    value       = 31,
    min         = 0,
    max         = int(ticker_max_week['week'].max()),
    step        = 1,
    description = 'Min week ≥',
    continuous_update = False,          # update on mouse-up only
    style   = {'description_width': '80px',
               'handle_color'    : '#3f88c5'},   # blue handle
    layout  = widgets.Layout(width='600px')      # wider track
)

import matplotlib.pyplot as plt

def plot_avg_brier(min_week):
    # Filter tickers with max week > min_week
    filtered = ticker_max_week[ticker_max_week['week'] > min_week]
    tickers = filtered['ticker'].unique()
    df = data[data['ticker'].isin(tickers)]
    
    # Calculate Brier score: (prob - resolution)^2
    df = df.copy()
    df['brier'] = (df['prob'] - df['resolution']) ** 2

    # Group by week and calculate average Brier
    avg_brier = df.groupby('week')['brier'].mean().reset_index()

    # Only show weeks >= min_week
    avg_brier = avg_brier[avg_brier['week'] <= min_week]

    # Calculate mean and 95% confidence interval for Brier score by week
    summary = df.groupby('week')['brier'].agg(['mean', 'count', 'std']).reset_index()
    summary['sem'] = summary['std'] / summary['count']**0.5
    summary['ci95_hi'] = summary['mean'] + 1.96 * summary['sem']
    summary['ci95_lo'] = summary['mean'] - 1.96 * summary['sem']

    summary['ci9_hi'] = summary['mean'] + 1.96 * summary['std']
    summary['ci9_lo'] = summary['mean'] - 1.96 * summary['std']

    plt.figure(figsize=(8,5))
    sns.lineplot(x='week', y='mean', data=summary, marker='o', label='Average Brier')
    plt.fill_between(summary['week'], summary['ci95_lo'], summary['ci95_hi'], alpha=0.3, label='95% CI')
    plt.fill_between(summary['week'], summary['ci9_lo'], summary['ci9_hi'], alpha=0.3, label='95% CI')

    plt.gca().invert_xaxis()  # Reverse x-axis
    plt.xlabel('Week')
    plt.ylabel('Average Brier Score')
    plt.title(f'Average Brier Score by Week (tickers with max week > {min_week})')
    plt.xlim(left=min_week)
    plt.legend()
    plt.show()

interact(plot_avg_brier, min_week=pretty_slider)

interactive(children=(IntSlider(value=31, continuous_update=False, description='Min week ≥', layout=Layout(wid…

<function __main__.plot_avg_brier(min_week)>